In [9]:
!pip install openai
import pandas as pd
from datetime import datetime
import os
from openai import OpenAI
secret_key = ""
client = OpenAI(api_key=secret_key)

In [10]:
class Agent:
    """A class to represent an agent that processes tasks based on specific characteristics.

    Attributes:
        model (str): The model version to use for processing.
        task_description (str): The description of the task to be processed.
        persona_prefix (str): The prefix to add before the persona description.
        persona_suffix (str): The suffix to add after the persona description.
        persona (str): The persona description to adopt for the task.
        system_instructions (str): The complete system instructions including persona and constraints.
        original_task_description (str): The original task description without modifications.
        current_task_description (str): The current task description that may include modifications.

    Args:
        task_description (str): The description of the task to be processed.
        data (pd.DataFrame, optional): The dataset to use for generating persona descriptions.
        persona_mapping (dict, optional): Mapping to convert dataset rows into persona descriptions.
        ideology (str, optional): The ideological filter to apply when selecting data for persona generation.
        query_str (str, optional): A string that is used for a pandas query clause on the dataframe
        model (str): The model version to use for processing. Defaults to "gpt-3.5-turbo".
        system_instructions (str): Additional instructions for the system. Defaults to an empty string.
        persona (str): Predefined persona description. If not provided, it will be generated based on data and ideology.
        persona_prefix (str): The prefix for the system instructions. Defaults to a standard instruction about adopting a persona.
        persona_suffix (str): The suffix for the system instructions. Defaults to standard constraints to follow.

    Methods:
        process_task(previous_response=""): Process the task, optionally building upon a previous response.
        get_persona_description(data, ideology): Generates a persona description based on the dataset and ideology.
        filter_data_by_ideology(data, ideology): Filters the dataset based on the specified ideology.
        _get_response(task): Internal method to interact with the API and get a response.
        row2persona(row, persona_mapping): Converts a dataset row into a persona description string.
    """
    def __init__(self,
                 task_description,
                 data=None,
                 persona_mapping = None,
                 ideology=None,
                 query_str = None,
                 model="gpt-3.5-turbo",
                 system_instructions="",
                 persona="",
                 persona_prefix="INSTRUCTIONS\nWhen answering questions or performing tasks, always adopt the following persona.\n\nPERSONA:\n",
                 persona_suffix="\n\nCONSTRAINTS\n- When answering, do not disclose your partisan or demographic identity in any way.\n- Think, talk, and write like your persona.\n- Use plain language.\n- Adopt the characteristics of your persona.\n-Do not be overly polite or politically correct."):
        """Initialize an agent with specific characteristics and dataset."""
        self.model = model
        self.persona_mapping = persona_mapping
        self.task_description = task_description
        self.persona_prefix = persona_prefix
        self.persona_suffix = persona_suffix
        self.persona = persona
        self.ideology = ideology
        self.data = data
        self.query_str = query_str
        self.original_task_description = task_description # Original task description
        self.current_task_description = task_description # This will be updated when it's reading in from another agent
        self.validate()
        if not self.persona:
            if ideology:
              self.persona = self.get_persona_description_ideology(data, ideology)
            else:
              self.persona = self.row2persona(data.query(query_str).sample(1).iloc[0], self.persona_mapping)
        self.system_instructions = self.persona_prefix + self.persona + self.persona_suffix

    def process_task(self, previous_response=""):
        """Process the task, optionally building upon a previous response."""
        task = self.original_task_description
        if previous_response:
          task = f"""
{task}
INCORPORATE PRIOR ANSWERS
- Here is what was previously said: '''{previous_response}'''
- Do not respond directly to what was previously said, but keep the best points from what was previously said. Ensure the perspective from prior responses are represented in your balanced answer.
"""
        self.current_task_description = task
        return self._get_response(task)


    def get_persona_description_ideology(self, data, ideology):
        """Generates a persona description based on the dataset and ideology."""
        filtered_data = self.filter_data_by_ideology(data, ideology)
        if not filtered_data.empty:
            selected_row = filtered_data.sample(n=1).iloc[0]
            return self.row2persona(selected_row, self.persona_mapping)
        return "No data available for the specified ideology."

    def filter_data_by_ideology(self, data, ideology):
        """Filters the dataset based on the specified ideology."""
        if ideology == 'liberal':
            return data[data['ideo5'].isin([1, 2])]
        elif ideology == 'conservative':
            return data[data['ideo5'].isin([4, 5])]
        return data

    def _get_response(self, task):
        """Internal method to interact with the API and get a response."""
        try:
            completion = client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": self.system_instructions},
                    {"role": "user", "content": task}
                ]
            )
            return completion.choices[0].message.content
        except Exception as e:
            print("Error in _get_response:", e)
            return None


    @staticmethod
    def row2persona(row, persona_mapping):
        """Converts a dataset row into a persona description string."""
        persona_description_str = ""
        for key, value in persona_mapping.items():
            field_name = value['name']
            if key == 'race':
              race_found = False
              for race_key in ['eth', 'rwh', 'rbl', 'rain', 'ras', 'rpi', 'roth']:
                  if race_key in row and str(row[race_key]) == '1':
                      race_description = value['values'][race_key].get('1', 'Unknown')
                      if race_description:  # If a description exists for the '1' value
                          persona_description_str += f"{field_name} {race_description}. "
                          race_found = True
                          break  # Stop checking once a race is found
              if not race_found:
                  # If no race was indicated or if the race data does not match expected values
                  persona_description_str += f"{field_name} Unknown. "
            elif key in row:
                if key == "birthyr_dropdown" and 'integer' in value['values']:
                    age = datetime.now().year - row[key]
                    persona_description_str += f"{field_name} {age}. "
                else:
                    mapped_value = value['values'].get(str(row[key]), 'Unknown')
                    if "inapplicable" not in mapped_value.lower() and "legitimate skip" not in mapped_value.lower() and "unknown" not in mapped_value.lower():
                      persona_description_str += f"{field_name} {mapped_value}. "
        return persona_description_str.lower()

    def validate(self):
      assert self.original_task_description is not None, "Need to provide some task instructions"
      if self.ideology or self.query_str:
        assert self.data is not None and self.persona_mapping is not None, "If you use either `ideology' or `query_str' you need to provide both a dataframe and a persona mapping to process rows of the dataframe."
